In [59]:
import xml.etree.ElementTree as ET
import pandas as pd
import cv2
from pathlib import Path

In [60]:
# xml_file = 'test-133816-dont-annotate.xml'
# bbox_csv_file = 'bbox.csv'
DATADIR = '/media/aubrey/9C33-6BBD/06-23-test-cut-detector-20'
CVATXMLFILE = '06-23-test-cut-detector-20.xml'
#bbox_csv_file = '/media/aubrey/9C33-6BBD/06-23-test-cut-detector-20/bbox.csv'
VIDEOFILE = '20200623_processed.mp4'

In [61]:
def str2int(s):
    return int(float(s))

# str2int('7.53')

In [62]:
dfbb

,frame,keyframe,xtl,ytl,xbr,ybr,track_id,label
0,134,1,1879,739,1895,768,0,cut
1,135,1,1813,739,1828,768,0,cut
2,136,1,1755,741,1771,770,0,cut
3,137,1,1694,736,1710,765,0,cut
4,138,1,1630,728,1646,758,0,cut
...,...,...,...,...,...,...,...,...
109,174,1,429,458,477,505,9,cut
110,175,1,375,421,423,468,9,cut
111,176,1,295,367,364,429,9,cut
112,177,1,220,301,290,359,9,cut


In [63]:
def get_bounding_boxes(xml_file):
    """
    Returns a pandas dataframe.
    """   
    tree = ET.parse(xml_file)
    root = tree.getroot()

    mylist = []
    for track in root.findall('track'):
        for box in track.findall('box'):
            mydict = box.attrib
            mydict.update(track.attrib)
            mylist.append(mydict)
    df = pd.DataFrame(mylist)
    df = df[(df.occluded=='0') & (df.outside=='0')]
    df.drop(['occluded', 'outside'], axis=1, inplace=True)
    df.rename({'id': 'track_id'}, axis=1, inplace=True)
    df.xbr = df.xbr.apply(lambda x: str2int(x))
    df.xtl = df.xtl.apply(lambda x: str2int(x))
    df.ybr = df.ybr.apply(lambda x: str2int(x))
    df.ytl = df.ytl.apply(lambda x: str2int(x))
    df.frame = df.frame.apply(lambda x: str2int(x))
    df.track_id = df.track_id.apply(lambda x: str2int(x))
    return df

#get_bounding_boxes(xml_file)

In [64]:
def write_bounding_box_image(row):
    """
    Writes image contained in bounding box.
    """
    cap = cv2.VideoCapture(f'{DATADIR}/{VIDEOFILE}')
    cap.set(1, row.frame)
    _, frame = cap.read()
    img = frame[row.ytl:row.ybr, row.xtl:row.xbr]
    cap.release()
    prefix = f'{DATADIR}/tracks/track{row.track_id:04d}'
    Path(prefix).mkdir(parents=True, exist_ok=True)
    cv2.imwrite(f'{prefix}//t{row.track_id:04d}f{row.frame:04d}.jpg', img)

# MAIN

In [65]:
dfbb = get_bounding_boxes(f'{DATADIR}/{CVATXMLFILE}')

grouped = dfbb.groupby('track_id')
for track_id, df in grouped:
    df.apply(write_bounding_box_image, axis=1)    